In [1]:
import pandas as pd
import torch
import re
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from utils.utils_train import train_supervised, train_models_in_threads, test_model_in_batches
from utils.utils_plots import plot_interesting_features, plot_metric_data, save_figure, plot_cluster_data
from utils.utils_dataset import prepare_dataset, balance_data_for_clients
from utils.utils_dataset import display_dataset_split
from utils.utils_metrics import calculate_metrics, plot_confusion_matrix, calculate_roc_auc
import threading
import itertools
import matplotlib.pyplot as plt
import pandas as pd
import concurrent.futures
import os
import time


C:\Users\Miha\AppData\Local\Temp\ipykernel_3344\3119565666.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from model.eFedGauss import eFedGauss

In [3]:
# Load the dataset
file_path = 'creditcard.csv' #Download from https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
data = pd.read_csv(file_path)
feature_dim = 29

# Remove the first dimension/column
data = data.drop(data.columns[0], axis=1)

# Compute the ranges
ranges = data.max() - data.min()

# Display the ranges
print(ranges)


V1           58.862440
V2           94.773457
V3           57.708148
V4           22.558515
V5          148.544973
V6           99.462131
V7          164.146736
V8           93.223927
V9           29.029061
V10          48.333399
V11          16.816387
V12          26.532107
V13          12.918764
V14          29.741092
V15          13.376686
V16          31.444966
V17          34.416326
V18          14.539815
V19          12.805499
V20          93.918625
V21          62.033221
V22          21.436234
V23          67.336147
V24           7.421176
V25          17.814986
V26           6.121896
V27          54.177877
V28          49.277892
Amount    25691.160000
Class         1.000000
dtype: float64


In [4]:
print(f"{torch.cuda.is_available()}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #torch.device("cpu") #

False


In [5]:
#Experiment to determine best problem-specific meta-parameters
flag_settings_experiment = 0
if flag_settings_experiment:

    num_clients = 1

    # Define the range of values for each parameter
    num_sigma_values = [5, 10, 15, 20]
    kappa_join_values = [0.3, 0.5, 0.8]
    N_r_values = [10, 20, 30]

    # Total number of experiments
    total_experiments = len(num_sigma_values) * len(kappa_join_values) * len(N_r_values)
    completed_experiments = 0

    # Define other parameters and data setup
    local_model_params = {
        "feature_dim": feature_dim,
        "num_classes": 2,
        "kappa_n": 1,
        "S_0": 1e-10,
        "c_max": 100,
        "device": device
    }

    # Placeholder for the best parameters and best score
    best_params = None
    best_score = 0

    # List to store all results
    results = []

    # Function to write data to a file
    def write_to_file(file_path, data, mode='a'):
        with open(file_path, mode) as file:
            file.write(data + "\n")

    # Prepare the dataset
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    client_train_unbalanced, test_data, all_data = prepare_dataset(X, y, num_clients)
    client_train = balance_data_for_clients(client_raw_data=client_train_unbalanced, balance=["random"], local_models=None, round=1)
    
    # Initialize a lock and a shared variable for progress tracking
    lock = threading.Lock()
    completed_experiments = 0
    total_experiments = len(num_sigma_values) * len(kappa_join_values) * len(N_r_values)

    # Function to execute model training and evaluation
    def train_evaluate_model(params):
        global completed_experiments
        
        num_sigma, kappa_join, N_r = params
        local_model_params.update({"num_sigma": num_sigma, "kappa_join": kappa_join, "N_r": N_r})

        local_model = eFedGauss(**local_model_params)
        train_supervised(local_model, client_train[0])

        _, pred_max, _ = test_model_in_batches(local_model, test_data, batch_size = 1000)
        metrics = calculate_metrics(pred_max, test_data, weight="binary")

        result_str = f"num_sigma:{num_sigma}, kappa_join:{kappa_join}, N_r:{N_r}, f1_score:{metrics['f1_score']}, precission:{metrics['precision']}, recall:{metrics['recall']}"
        print(result_str)
        write_to_file("experiment_results.txt", result_str)  # Write results to file
        
        # Update progress
        with lock:
            completed_experiments += 1
            progress = (completed_experiments / total_experiments) * 100
            print(f"Progress: {completed_experiments}/{total_experiments} ({progress:.1f}%)")

        return {"num_sigma": num_sigma, "kappa_join": kappa_join, "N_r": N_r, "f1_score": metrics["f1_score"], "precission": metrics["precision"], "recall": metrics["recall"]}
        

    # Write initial setup data to file
    initial_setup_str = f"Initial Setup: num_clients={num_clients}, num_sigma_values={num_sigma_values}, kappa_join_values={kappa_join_values}, N_r_values={N_r_values}"
    write_to_file("experiment_results.txt", initial_setup_str, mode='w')  # 'w' to overwrite if exists

    # Using ThreadPoolExecutor to run in multiple threads
    with concurrent.futures.ThreadPoolExecutor() as executor:
        param_combinations = list(itertools.product(num_sigma_values, kappa_join_values, N_r_values))
        results = list(executor.map(train_evaluate_model, param_combinations))

    results_df = pd.DataFrame(results)


In [6]:
def compare_models(model1, model2):
    differences = []

    # Function to find differing indices within the overlapping range
    def find_differing_indices(tensor1, tensor2):
        min_length = min(tensor1.size(0), tensor2.size(0))
        differing = (tensor1[:min_length] != tensor2[:min_length]).nonzero(as_tuple=False)
        if differing.nelement() == 0:
            return "No differences"
        else:
            return differing.view(-1).tolist()  # Flatten and convert to list

    # Compare mu parameter and find differing indices
    mu_equal = torch.equal(model1.mu[:model1.c], model2.mu[:model2.c])
    if not mu_equal:
        differing_indices_mu = find_differing_indices(model1.mu[:model1.c], model2.mu[:model2.c])
        differences.append(f"mu parameter differs at indices {differing_indices_mu}")

    # Compare S parameter and find differing indices
    S_equal = torch.equal(model1.S[:model1.c], model2.S[:model2.c])
    if not S_equal:
        differing_indices_S = find_differing_indices(model1.S[:model1.c], model2.S[:model2.c])
        differences.append(f"S parameter differs at indices {differing_indices_S}")

    # Compare n parameter and find differing indices
    n_equal = torch.equal(model1.n[:model1.c], model2.n[:model2.c])
    if not n_equal:
        differing_indices_n = find_differing_indices(model1.n[:model1.c], model2.n[:model2.c])
        differences.append(f"n parameter differs at indices {differing_indices_n}")

    # Check if there are any differences
    if differences:
        difference_str = ", ".join(differences)
        return False, f"Differences found in: {difference_str}"
    else:
        return True, "Models are identical"


In [7]:
#Helper function for saving the experiments
def write_to_file(file_path, data, mode='a'):
    with open(file_path, mode) as file:
        file.write(data + "\n")

In [8]:

# This function handles the learning of the federated model, one experiment
def run_individual_experiment(federated_model_params, local_model_params, num_clients, num_rounds, client_raw_train, test_data, balance_techniques, test_clients=True):

    # Initialize a model for each client
    local_models = [eFedGauss(**local_model_params) for _ in range(num_clients)]
    federated_model = eFedGauss(**federated_model_params)

    # Initialize a list to store the metrics for each round
    round_metrics = []
    client_train = []

    result_file = "experiment_results.txt"
  
    # Execute communication loop
    for round in range(num_rounds):

        start = time.time() #For debugging

        print(f"\n --- \n --- Communication Round {round + 1} ---")
        round_info = f"--- Communication Round {round + 1} ---\n---\n"
        
        # Undersample client data in each round 
        client_train = balance_data_for_clients(client_raw_data=client_raw_train, balance_techniques=balance_techniques, local_models=local_models, round=round)
        display_dataset_split(client_train, test_data)

        # Train local models in parallel threads
        train_models_in_threads(local_models, client_train)

        # Update federated model with local models
        for client_idx, _ in enumerate(local_models):

            print(f"\n Updating agreggated model with client {client_idx + 1}")
            print(f"Number of local model clusters = {sum(local_models[client_idx].n[0:local_models[client_idx].c] > 0)}")
            local_models[client_idx].federal_agent.federated_merging()
            print(f"Number of local model clusters after merging = {sum(local_models[client_idx].n[0:local_models[client_idx].c] > 0)}")

            # Calculate and collect metrics for each client model
            # The testing was moved further, to after the federated model is returned to the clients
            #_, client_pred, _ = test_model_in_batches(local_models[client_idx], client_train[client_idx], batch_size=500)
            #client_binary = calculate_metrics(client_pred, client_train[client_idx], "binary")
            #print(f"Train Metrics: {client_binary}")
            #plot_confusion_matrix(pred_max, clients_data[client_idx])


            federated_model.federal_agent.merge_model_privately(local_models[client_idx], local_models[client_idx].kappa_n, pred_min = 0)
            federated_model.federal_agent.federated_merging()
            
            print(f"Federated clusters after merging = {sum(federated_model.n[0:federated_model.c]> federated_model.kappa_n)}")
                
        print("\n")
    
        # Evaluate federated model
        fed_scores, fed_pred, _ = test_model_in_batches(federated_model, test_data, batch_size=1000)
        fed_binary = calculate_metrics(fed_pred, test_data, "binary")
        fed_roc_auc = calculate_roc_auc(fed_scores, test_data)
        print(f"Test Metrics: {fed_binary}")
        print(f"Test ROC AUC: {fed_roc_auc}")
        #plot_confusion_matrix(pred_max_fed, test_data) #Mainly for debugging

        print("\n")

        # Return the updated federated model to each client
        client_metrics = [] # Reset client metrics for the new round
        for client_idx, _ in enumerate(local_models):
            print(f"Returning updated model to client {client_idx + 1}")
            
            # Normalize the covariance matrices and reset the number of samples
            # This is required as the number of samples increases exponentially with communication
            with torch.no_grad():    
                local_models[client_idx].S.data /= torch.min(local_models[client_idx].n[:local_models[client_idx].c].data.clone())
                local_models[client_idx].n.data /= torch.min(local_models[client_idx].n[:local_models[client_idx].c].data.clone())
                local_models[client_idx].S_glo.data /= torch.sum(local_models[client_idx].n_glo).unsqueeze(-1)
                local_models[client_idx].n_glo = torch.ones_like(local_models[client_idx].n_glo)

            #Return federated model back to the clients
            local_models[client_idx].federal_agent.merge_model_privately(federated_model, 0, pred_min = 0)
            #local_models[client_idx].federal_agent.federated_merging() #This could be useful but it is somewhat redundant
            print(f"Number of local model clusters after transfer = {sum(local_models[client_idx].n[0:local_models[client_idx].c]> 0)}")

            # Testing the clients also, for debugging and illustration plot
            if test_clients:

                # Calculate and collect metrics for each client model
                _, client_pred, _ = test_model_in_batches(local_models[client_idx], test_data, batch_size=500)
                client_binary = calculate_metrics(client_pred, test_data, "binary")
                print(f"Test Metrics client {client_idx} after merge: {client_binary}")
                # plot_confusion_matrix(pred_max, clients_data[client_idx])
                
                # Calculate additional metrics for each client
                client_metrics.append({
                    'client_idx': client_idx,
                    'binary': client_binary,
                    'clusters': sum(local_models[client_idx].n[0:local_models[client_idx].c].cpu()> 0)
                })
            
            #local_models[client_idx].score = torch.ones_like(local_models[client_idx].score)
            local_models[client_idx].num_pred = torch.ones_like(local_models[client_idx].num_pred)
            local_models[client_idx].age = torch.ones_like(local_models[client_idx].age)

        # Reset the number of samples per cluster
        # The number of clusters increases exponentially if this is not handled
        with torch.no_grad():     
            federated_model.S[:federated_model.c].data /= torch.min(federated_model.n[:federated_model.c].data.clone())
            federated_model.n[:federated_model.c].data /= torch.min(federated_model.n[:federated_model.c].data.clone())
            federated_model.S_glo.data /= torch.sum(federated_model.n_glo).unsqueeze(-1)
            federated_model.n_glo = torch.ones_like(federated_model.n_glo)
        
        #federated_model.num_pred = torch.ones_like(federated_model.num_pred) #Future work, evaluation of accuracy

        # Print and write round information to file
        round_info = f"--- End of Round {round + 1} ---\n"
        print(round_info)

        # Save the results of this round of communication
        round_metrics.append({
            'round': round + 1,
            'federated_model': {
                'clusters': sum(federated_model.n[0:federated_model.c].cpu() > federated_model.kappa_n),
                'binary': fed_binary,
                'roc_auc': fed_roc_auc
            },
            'client_metrics': client_metrics
        })
        
        # Plot features for the current round
        plt.close('all')  # Close all existing plots to free up memory
        if 0:
            #fig1 = plot_interesting_features(client_train[0], model=federated_model, num_sigma=federated_model.num_sigma, N_max=federated_model.kappa_n)   
            #save_figure(fig1, "./Images/credit_fraud_clusters", format='pdf')
            fig2 = plot_interesting_features(client_train[0], model=federated_model, num_sigma=2, N_max=federated_model.kappa_n)   
            save_figure(fig2, ".Images/credit_fraud_samples.pdf", format='pdf')

        # Iterate over each round's metrics and write to file
        for metric in round_metrics:
            metric_info = f"Round {metric['round']}: Metrics: {metric['federated_model']['binary']}, ROC AUC: {metric['federated_model']['roc_auc']}\n"
            print(metric_info)  # Print each round's metrics
            try:
                write_to_file(result_file, metric_info)  # Write to file, for debugging
            except:
                print("Could not write to file.")
                pass
        
        # Save the experiment intermediate results for progress checking and debugging
        experiment_file = f".Results/mid_experiment_{federated_model_params['num_sigma']}_{int(federated_model_params['kappa_join']*10)}_{federated_model_params['N_r']}_{num_clients}_{len(balance_techniques)}_{federated_model_params['c_max']}.pth"
        torch.save(round_metrics, experiment_file)
        print(f"Saved experiment to {experiment_file}")
        print(f"Round time was {(time.time() - start):.1f}s")

    # After all rounds print and save the metrics
    final_info = "All Rounds Completed. Metrics Collected:\n"
    print(final_info)

    # Iterate over each round's metrics and write to file
    for metric in round_metrics:
        metric_info = f"Round {metric['round']}: "
        metric_info += f"Federated Model - Clusters: {metric['federated_model']['clusters']}, "
        metric_info += f"Binary Metrics: {metric['federated_model']['binary']}, ROC AUC: {metric['federated_model']['roc_auc']}\n"

        for client_metric in metric['client_metrics']:
            metric_info += f"Client {client_metric['client_idx']} - Binary: {client_metric['binary']}\n"

        print(metric_info)  # Print each round's metrics
        #write_to_file(result_file, metric_info)  # Write to file

    return round_metrics


In [9]:
# Experiment to determine best settings for furhter experimentation
# Define the range of values for each parameter
num_sigma_values = [10, 15, 20]
kappa_join_values = [0.3, 0.4, 0.5, 0.8, 1, 2]
N_r_values = [15, 20, 25, 30]
proportion = [5, 3, 1]

# List of client counts and data configuration indices
client_counts = [3]

# Number of communication rounds
num_rounds = 5

In [10]:

# Sets all experiments
def run_experiments(data, client_counts, federated_model_params, local_model_params, num_rounds, proportions, flag_profiler=False, flag_test_clients=False):
   
    experiments = [] # Experiment metrics
    results_dir = ".Results"  # Directory to save the results
    os.makedirs(results_dir, exist_ok = True)  # Create the directory if it doesn't exist

    for num_clients in client_counts:
        for proportion in proportions:

            # Print all experiment settings
            print(f'\n *** Experiment details ***:\n' 
            f'  - Number of Clients: {num_clients}\n' 
            f'  - Number of Rounds: {num_rounds}\n'
            f'  - Number of Clusters: {federated_model_params["c_max"]}\n'
            f'  - Data dristribution proportions: {proportion}\n'
            f'  - num_sigma Parameter: {federated_model_params["num_sigma"]}\n'
            f'  - kappa_join Parameter: {federated_model_params["kappa_join"]}\n'
            f'  - N_r Parameter: {federated_model_params["N_r"]}')
   
            # Create train and test dataset
            X = data.iloc[:, :-1].values
            y = data.iloc[:, -1].values
            client_raw_train, test_data, all_data = prepare_dataset(X, y, num_clients)

            # Set undersampling techniques
            balance_techniques = ['random'] * int(proportion)

            if flag_profiler:

                # Import profiling packages only if profiler is True
                import cProfile
                import yappi
                try:
                    # Try to load the memory_profiler extension if it's available
                    get_ipython().run_line_magic('load_ext', 'memory_profiler')
                except:
                    print("Memory profiler not available.")

                print(f"... with profiler")
                pr = cProfile.Profile()
                pr.enable()
                yappi.start()
                
                #Run one experiment with profiller
                metrics = run_individual_experiment(federated_model_params=federated_model_params, 
                                                    local_model_params=local_model_params, 
                                                    num_clients=num_clients, num_rounds=num_rounds, 
                                                    client_raw_train=client_raw_train, test_data=test_data,
                                                    balance_techniques=balance_techniques, test_clients=flag_test_clients)
                
                yappi.stop()
                pr.disable()

                pr.print_stats(sort='cumtime')
                yappi.get_thread_stats().print_all()
                yappi.get_func_stats().print_all()

            else:

                #Run one experiment
                metrics = run_individual_experiment(federated_model_params=federated_model_params, 
                                                    local_model_params=local_model_params, 
                                                        num_clients=num_clients, num_rounds=num_rounds, 
                                                    client_raw_train=client_raw_train, test_data=test_data,
                                                    balance_techniques=balance_techniques, test_clients=flag_test_clients)
                
            #Construct a specific name for the saved file
            file_name = f'experiment_metrics_num_clients_{num_clients}_rounds_{num_rounds}_proportions_{proportion}_num_sigma_{federated_model_params["num_sigma"]}_kappa_join_{int(federated_model_params["kappa_join"]*10)}_N_r_{federated_model_params["N_r"]}_clusters_{federated_model_params["c_max"]}_Fscore_{1000*metrics[-1]["federated_model"]["binary"]["f1_score"]:.0f}.pth'
            file_path = os.path.join(results_dir, file_name)

            #Save the experiments
            torch.save(metrics, file_path)
            print(f"Saved experiments to {file_path}")
    return experiments

In [11]:

# Extracts the F1 score at the last communication round
def evaluate_metric(experiments, round_number):
    for experiment in experiments:
        if experiment[1][-1]['round'] == int(round_number): #[1] is the experiment values, [0] is the name of the experiment, [-1] is the last round
            return experiment[1][-1]['federated_model']['binary']['f1_score']
    return None

#Experiment with different model meta-parameters
def run_parameterized_experiments(data, client_counts, proportions, num_rounds):
    best_setting = None
    best_f1_score = -float('inf')
    all_experiments_metrics = []

    for num_sigma in num_sigma_values:
        for kappa_join in kappa_join_values:
            for N_r in N_r_values:

                    # Update the model parameters
                    federated_model_params.update({"num_sigma": num_sigma, "kappa_join": kappa_join, "N_r": N_r})
                    local_model_params.update({"num_sigma": num_sigma, "kappa_join": kappa_join, "N_r": N_r})

                    # Run the experiment
                    experiments = run_experiments(
                        data=data,
                        client_counts=client_counts,
                        federated_model_params=federated_model_params, 
                        local_model_params=local_model_params, 
                        num_rounds=num_rounds, 
                        proportions=proportions, 
                        flag_profiler= False,
                        flag_test_clients = False)

                    # Store all experiments' metrics
                    all_experiments_metrics.append(experiments)

                    # Evaluate the F1 score of the 5th round
                    f1_score = evaluate_metric(experiments, num_rounds)
                    
                    # Update the best setting if current setting's F1 score is better
                    if f1_score and f1_score > best_f1_score:
                        best_f1_score = f1_score
                        best_setting = (num_sigma, kappa_join, N_r, proportions)

    return best_setting, best_f1_score, all_experiments_metrics

In [12]:
#Experiment to determine the best settings 
if flag_settings_experiment:
    
    # Run the experiments to determine the best settings
    best_setting, best_f1_score, all_experiments_metrics = run_parameterized_experiments(data, client_counts, data_config_indices, num_rounds)
    print("Best Setting:", best_setting)
    print("Best F1 Score:", best_f1_score)

    # Save the best setting and all experiments' metrics
    torch.save({
        "best_setting": best_setting,
        "best_f1_score": best_f1_score,
        "experiments_metrics": all_experiments_metrics
    }, 'experiment_results.pth')


In [13]:
# List of client counts and data configuration indices
client_counts = [3, 10]

# Number of communication rounds
num_rounds = 31

# Data proportion for the experiments
proportions = [5, 5, 5, 10, 10, 10, 1, 1, 1]

# Model parameters
local_model_params = {
    "feature_dim": feature_dim,
    "num_classes": 2,
    "kappa_n": 1,
    "num_sigma": 15,
    "kappa_join": 0.4,
    "S_0": 1e-10,
    "N_r": 30,
    "c_max": 1000,
    "device": device
}

federated_model_params = {
    "feature_dim": feature_dim,
    "num_classes": 2,
    "kappa_n": 1,
    "num_sigma": 15,
    "kappa_join": 0.4,
    "S_0": 1e-10,
    "N_r": 30,
    "c_max": 300,
    "device": device
}

if flag_settings_experiment:
    # Update the model parameters with the best setting
    federated_model_params.update({
        "num_sigma": best_setting[0],
        "kappa_join": best_setting[1],
        "N_r": best_setting[2]
    })

    local_model_params.update({
        "num_sigma": best_setting[0],
        "kappa_join": best_setting[1],
        "N_r": best_setting[2]
    })

    # Set the number of rounds and proportion with the best setting
    proportion = best_setting[3]

# Run the experiments with the best setting for 30 rounds
experiments = run_experiments(
            data=data, 
            client_counts=client_counts,
            federated_model_params=federated_model_params, 
            local_model_params=local_model_params, 
            num_rounds=num_rounds, 
            proportions=proportions, 
            flag_profiler=False,
            flag_test_clients = False)



 *** Experiment details ***:
  - Number of Clients: 3
  - Number of Rounds: 31
  - Number of Clusters: 300
  - Data dristribution proportions: 5
  - num_sigma Parameter: 15
  - kappa_join Parameter: 0.4
  - N_r Parameter: 30
--- Communication Round 1 ---
Client 1: {0: 645, 1: 129}
Client 2: {0: 645, 1: 129}
Client 3: {0: 640, 1: 128}
Test Set: {0: 56856, 1: 106}

Combined Number of Samples per Class:
Class 0: 58786 samples
Class 1: 492 samples

Total Number of Samples Across All Datasets: 59278
Debugging has been disabled.
Evolving has been enabled.
Debugging has been disabled.
Evolving has been enabled.
Debugging has been disabled.
Evolving has been enabled.
Number of local model clusters = 282
Updating agreggated model with client 1
Updated var_glo values: tensor(14.7428)
Federated clusters after merging = 92
Number of local model clusters = 311
Updating agreggated model with client 2
Updated var_glo values: tensor(14.9009)
Federated clusters after merging = 192
Number of local mode

KeyboardInterrupt: 

In [ ]:
#Select experiment for plotting
name = "experiment_metrics_num_clients_3_rounds_31_proportions_10_num_sigma_15_kappa_join_4_N_r_30_clusters_1000_Fscore_64.pth"
experiments = torch.load(f".Results/{name}")            

In [ ]:
# Show experiment metrics over the communication rounds
figs_metrics = []
figs_clusters = []
experiment_name = []
rounds_max = 30
if len(name) > 40:
        metrics = experiments
        rounds = [m['round'] for m in metrics]
        if rounds[-1] > rounds_max:
             rounds = rounds[:rounds_max]
             metrics = metrics[:rounds_max]
        # Plot and collect figures
       
        figs_metrics.append(plot_metric_data(metrics, ['f1_score', 'precision', 'recall'], rounds, ''))
        figs_clusters.append(plot_cluster_data(metrics, rounds))
        print(f"{100*metrics[-1]['federated_model']['binary']['f1_score']:.0f}")
else:
        rounds = [m['round'] for m in experiments]
        if rounds[-1] > rounds_max:
             rounds = rounds[:rounds_max]
             experiments = experiments[:rounds_max]

        # Plot and collect figures
        figs_metrics.append(plot_metric_data(experiments, ['f1_score', 'precision', 'recall'], rounds,"", legend=experiments))
        figs_clusters.append(plot_cluster_data(experiments, rounds, legend=experiments))
        print(f"{100*experiments[-1]['federated_model']['binary']['f1_score']:.0f}")

# Save figures from fig_metrics
for i, figure in enumerate(figs_metrics):
    save_path = f".Images/credit_fraud_metrics_{name}.pdf"
    save_figure(figure, save_path, "pdf")

# Save figures from fig_clusters
for i, figure in enumerate(figs_clusters):
    save_path = f".Images/credit_fraud_clusters_{name}.pdf"
    save_figure(figure, save_path, "pdf")

In [ ]:
# Function to extract settings from filename
def extract_settings(filename):
    pattern = r"(num_clients|rounds|proportions|num_sigma|kappa_join|N_r|clusters)_([0-9]+)"
    matches = re.findall(pattern, filename)
    return {match[0]: int(match[1]) for match in matches}

# Load all experiments and group by settings
experiments_by_settings = {}
directory = ".Results"
round_of_interest = 30  # Specify the round of interest
num_experiments = 3  # Specify the number of experiments required for each setting

for filename in os.listdir(directory):
    if filename.startswith("experiment_metrics_"):
        settings = extract_settings(filename)
        settings_key = tuple(sorted(settings.items()))

        if settings_key not in experiments_by_settings:
            experiments_by_settings[settings_key] = []

        experiment_data = torch.load(f"{directory}/{filename}")
        experiments_by_settings[settings_key].append(experiment_data)

# Calculate average and std of metrics at the specified round for each settings group
metrics_by_settings = {}

for settings, experiments in experiments_by_settings.items():
    if len(experiments) >= num_experiments:
        f1_scores, precisions, recalls, roc_aucs = [], [], [], []

        for experiment in experiments:
            if len(experiment) >= round_of_interest:
                round_data = experiment[round_of_interest - 1]['federated_model']
                f1_scores.append(100*round_data['binary']['f1_score'])
                precisions.append(100*round_data['binary']['precision'])
                recalls.append(100*round_data['binary']['recall'])
                roc_aucs.append(round_data['roc_auc'])

        metrics_by_settings[settings] = {
            'avg_f1': np.mean(f1_scores), 'std_f1': np.std(f1_scores),
            'avg_precision': np.mean(precisions), 'std_precision': np.std(precisions),
            'avg_recall': np.mean(recalls), 'std_recall': np.std(recalls),
            'avg_roc_auc': np.mean(roc_aucs), 'std_roc_auc': np.std(roc_aucs)
        }

for settings, metrics in metrics_by_settings.items():
    settings_str = ', '.join(f"{key}={value}" for key, value in settings)
    print(f"Settings: {settings_str} -> Average F1 Score: {metrics['avg_f1']:.4f}, Standard Deviation: {metrics['std_f1']:.4f}, "
          f"Average Precision: {metrics['avg_precision']:.4f}, Standard Deviation: {metrics['std_precision']:.4f}, "
          f"Average Recall: {metrics['avg_recall']:.4f}, Standard Deviation: {metrics['std_recall']:.4f}, "
          f"Average ROC AUC: {metrics['avg_roc_auc']:.4f}, Standard Deviation: {metrics['std_roc_auc']:.4f}")


In [ ]:
def generate_latex_table(data):
    latex_table = "\\begin{table}[!t]\n"
    latex_table += "\\centering\n"
    latex_table += "\\setlength{\\tabcolsep}{4pt}\n"  # Changed tabcolsep to 5pt
    latex_table += "\\scriptsize\n"
    latex_table += "\\caption{Results of the Credit Card Fraud Detection}\n"
    latex_table += "\\begin{tabular}{ll|cccccccc}\n"
    latex_table += "\\toprule\n"
    # Changed header row to use bold font and mathematical symbols where necessary
    latex_table += "\\bf{Data} & $\\mathbf{\\kappa}_c$ & \\multicolumn{2}{c}{\\bf{Precision}(\\%)$\\uparrow$} & \\multicolumn{2}{c}{\\bf{Recall}(\\%)$\\uparrow$} & \\multicolumn{2}{c}{\\bf{F1 score}(\\%)$\\uparrow$} & \\multicolumn{2}{c}{\\bf{ROC AUC}$\\uparrow$} \\\\\n"
    latex_table += "\\cmidrule(lr){3-4} \\cmidrule(lr){5-6} \\cmidrule(lr){7-8} \\cmidrule(lr){9-10}\n"
    # Changed number headings to bold
    latex_table += "&  & \\bf{3} & \\bf{10} & \\bf{3} & \\bf{10} & \\bf{3} & \\bf{10} & \\bf{3} & \\bf{10} \\\\\n"
    latex_table += "\\midrule\n"

    # Find the best F1 score for 3 clients and 10 clients
    best_f1_3 = max(metrics['avg_f1'] for settings, metrics in data.items() if 'avg_f1' in metrics and dict(settings).get('num_clients') == 3)
    best_f1_10 = max(metrics['avg_f1'] for settings, metrics in data.items() if 'avg_f1' in metrics and dict(settings).get('num_clients') == 10)


    # Sort and group data by settings, excluding num_clients
    grouped_data = {}
    for settings, metrics in data.items():
        # Create a key for grouping, excluding 'num_clients'
        group_key = tuple((k, v) for k, v in settings if k != 'num_clients')
        if group_key not in grouped_data:
            grouped_data[group_key] = {}
        num_clients = dict(settings).get('num_clients')
        grouped_data[group_key][num_clients] = metrics

    # Sort grouped data by proportions and then by clusters
    sorted_grouped_data = sorted(grouped_data.items(), key=lambda x: (dict(x[0]).get('proportions', 0), dict(x[0]).get('clusters', 0)))
   
    last_proportion = None
    for settings, client_data in sorted_grouped_data:
        proportions = dict(settings).get('proportions', 'N/A')
        
        # Add a horizontal line when the proportion setting changes
        if last_proportion is not None and last_proportion != proportions:
            latex_table += "\\hline\n"
        last_proportion = proportions
        metrics_3_clients = client_data.get(3)
        metrics_10_clients =  client_data.get(10)

        # Function to format metric value or empty placeholder
        format_metric = lambda val: f"$ {val:.1f} $" if val is not None else "$ $"
        format_roc_auc = lambda val: f"$ {val:.3f} $" if val is not None else "$ $"
        
        # Extract and format the dataset value
        proportions = dict(settings).get('proportions', 'N/A')
        dataset = f"{proportions}:1" if proportions != 'N/A' else "N/A"
        clusters = 2*dict(settings).get('clusters', 'N/A')


        # Extract and format metrics for 3 clients
        if metrics_3_clients:
            avg_precision_3 = format_metric(metrics_3_clients['avg_precision'])
            avg_recall_3 = format_metric(metrics_3_clients['avg_recall'])
            avg_f1_3 = format_metric(metrics_3_clients['avg_f1'])
            avg_roc_auc_3 = format_roc_auc(metrics_3_clients['avg_roc_auc'])
        else:
            avg_precision_3 = avg_recall_3 = avg_f1_3 = avg_roc_auc_3 = "$ $"

        # Extract and format metrics for 10 clients
        if metrics_10_clients:
            avg_precision_10 = format_metric(metrics_10_clients['avg_precision'])
            avg_recall_10 = format_metric(metrics_10_clients['avg_recall'])
            avg_f1_10 = format_metric(metrics_10_clients['avg_f1'])
            avg_roc_auc_10 = format_roc_auc(metrics_10_clients['avg_roc_auc'])
        else:
            avg_precision_10 = avg_recall_10 = avg_f1_10 = avg_roc_auc_10 = "$ $"
            
        # Bold the best F1 scores
        avg_f1_3 = f"$\\textbf{{{metrics_3_clients['avg_f1']:.1f}}}$" if metrics_3_clients and metrics_3_clients['avg_f1'] == best_f1_3 else f"${metrics_3_clients['avg_f1']:.1f}$" if metrics_3_clients else "$ $"
        avg_f1_10 = f"$\\textbf{{{metrics_10_clients['avg_f1']:.1f}}}$" if metrics_10_clients and metrics_10_clients['avg_f1'] == best_f1_10 else f"${metrics_10_clients['avg_f1']:.1f}$" if metrics_10_clients else "$ $"

        latex_table += f"    {dataset} & {clusters} & {avg_precision_3} & {avg_precision_10} & {avg_recall_3} & {avg_recall_10} & {avg_f1_3} & {avg_f1_10} & {avg_roc_auc_3} & {avg_roc_auc_10}\\\\\n"

    latex_table += "\\bottomrule\n"
    latex_table += "\\multicolumn{10}{p{0.95\linewidth}}{\\tiny The first column (Data) shown the proportion of non-fraud to fraud samples in the training data, which was selected by random undersampling. The second column ($\kappa_c$) shows the maximum number of clusters of the federated model, before pruning was used. The evaluation metrics shown are averages for 3 repetitions.}"
    latex_table += "\\end{tabular}\n"
    latex_table += "\\label{tab:credit_fraud}\n"
    latex_table += "\\end{table}\n"

    return latex_table

print(generate_latex_table(metrics_by_settings))


In [ ]:
# Let's write the README content to a Python file as a string, preserving the formatting

readme_content = """
# eFedGauss: A Federated Approach to Fuzzy Multivariate Gaussian Clustering
**This paper is in the review process.**

## Overview
eFedGauss introduces a novel method for data clustering and classification in federated learning environments. ...

## Repository Structure

The eFedGauss project is organized as follows:

- `model/`: Contains the implementation of the eFedGauss algorithm and its associated operations.
  - `clustering_operations/`: Functions related to the clustering process.
  - `consequence_operations/`: Operations for handling the consequence output of the model.
  - `eFedGauss/`: The main eFedGauss algorithm implementation.
  - `federated_operations/`: Federated learning specific operations.
  - `math_operations/`: Mathematical functions used across the model (distance computation).
  - `merging_mechanism/`: Logic for merging clusters.
  - `model_operations/`: Core operations for managing the lifecycle of the clustering model.
  - `removal_mechanism/`: Methods for removing clusters.

- `utils/`: Utility scripts to support the main algorithm.
  - `utils_dataset/`: Utilities for data handling and preprocessing.
  - `utils_metrics/`: Metrics for evaluating the model's performance.
  - `utils_plots/`: Plotting functions to visualize the results.
  - `utils_tables/`: Functions to generate result tables.
  - `utils_train/`: Training and testing.

- `credit_card_fraud_experiment/`: Experimental setup and results for the credit card fraud detection.

- `iris_flower_experiment/`: Experimental setup and results for the Iris Flower classification.

- `synthetic_experiment/`: Experimental setup and results for testing with synthetic data.

## Getting Started

To get started with eFedGauss, follow the setup instructions below.

### Setting Up the Environment

**Create and activate a Conda environment using the following commands:**

\```bash
conda create -n eFedGauss python=3.10
conda activate eFedGauss
\```

### Installing PyTorch and Related Packages

**Install PyTorch and related packages with the following command:**

\```bash
conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia
\```

### Installing Additional Requirements

**Install the remaining requirements from the `requirements.txt` file:**

\```bash
pip install -r requirements.txt
\```

## Usage

**Run the experiments using the Jupyter notebooks provided:**

- `credit_card_fraud_experiment.ipynb`: For credit card fraud detection.
- `iris_flower_experiment.ipynb`: For Iris Flower classification.
- `synthetic_experiment.ipynb`: For synthetic data experiments.

## Requirements

The required packages are listed in the `requirements.txt` file.

## Contribution

Contributions are welcome. Please follow the standard fork-and-pull request workflow.

## License

eFedGauss is under GNU General Public License v3 (GPLv3). See `LICENSE` for more details.
"""

# Save to a .py file
file_path = 'README.md'
with open(file_path, 'w') as file:
    file.write(readme_content)

file_path
